In [1]:
%load_ext autoreload
%autoreload 2
import sys,os
import torch
current_directory = os.getcwd()
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(os.path.abspath(os.path.dirname(current_directory))))))
from datasets.weather_bench import WeatherDataset

In [2]:
device = ("cuda" if torch.cuda.is_available() else "cpu" )
device = torch.device(device)

weather = WeatherDataset(1, device=device, normalize=True)
# dataset.shape:  torch.Size([7309, 100, 1450])
original = weather.load()

데이터셋 불러오는 중...
==== LOAD DATASET ====
 <xarray.Dataset>
Dimensions:                   (time: 7309, latitude: 28, longitude: 28,
                               level: 13)
Coordinates:
  * latitude                  (latitude) float32 39.0 38.75 38.5 ... 32.5 32.25
  * level                     (level) int64 50 100 150 200 ... 700 850 925 1000
  * longitude                 (longitude) float32 124.2 124.5 ... 130.8 131.0
  * time                      (time) datetime64[ns] 2011-12-31 ... 2016-12-31
Data variables: (12/14)
    10m_u_component_of_wind   (time, latitude, longitude) float32 ...
    10m_v_component_of_wind   (time, latitude, longitude) float32 ...
    2m_temperature            (time, latitude, longitude) float32 ...
    geopotential              (time, level, latitude, longitude) float32 ...
    mean_sea_level_pressure   (time, latitude, longitude) float32 ...
    sea_surface_temperature   (time, latitude, longitude) float32 ...
    ...                        ...
    total_cloud

Processing futures: 0it [00:00, ?it/s]

Processing futures: 13it [00:01, 10.76it/s]
Processing futures: 2it [00:02,  1.10s/it]


3.74402 sec
==== LOAD DATASET ====
 <xarray.Dataset>
Dimensions:                   (time: 7309, longitude: 14, latitude: 33,
                               level: 13)
Coordinates:
  * latitude                  (latitude) float64 21.0 22.5 24.0 ... 67.5 69.0
  * level                     (level) int64 50 100 150 200 ... 700 850 925 1000
  * longitude                 (longitude) float64 121.5 123.0 ... 139.5 141.0
  * time                      (time) datetime64[ns] 2011-12-31 ... 2016-12-31
Data variables: (12/14)
    10m_u_component_of_wind   (time, longitude, latitude) float32 ...
    10m_v_component_of_wind   (time, longitude, latitude) float32 ...
    2m_temperature            (time, longitude, latitude) float32 ...
    geopotential              (time, level, longitude, latitude) float32 ...
    mean_sea_level_pressure   (time, longitude, latitude) float32 ...
    sea_surface_temperature   (time, longitude, latitude) float32 ...
    ...                        ...
    total_cloud_cove

Processing futures: 13it [00:00, 18.45it/s]
Processing futures: 2it [00:01,  1.87it/s]


2.02509 sec
==== LOAD DATASET ====
 <xarray.Dataset>
Dimensions:                   (time: 7309, longitude: 17, latitude: 12,
                               level: 13)
Coordinates:
  * latitude                  (latitude) float64 2.903 8.71 ... 60.97 66.77
  * level                     (level) int64 50 100 150 200 ... 700 850 925 1000
  * longitude                 (longitude) float64 90.0 95.62 ... 174.4 180.0
  * time                      (time) datetime64[ns] 2011-12-31 ... 2016-12-31
Data variables: (12/14)
    10m_u_component_of_wind   (time, longitude, latitude) float32 ...
    10m_v_component_of_wind   (time, longitude, latitude) float32 ...
    2m_temperature            (time, longitude, latitude) float32 ...
    geopotential              (time, level, longitude, latitude) float32 ...
    mean_sea_level_pressure   (time, longitude, latitude) float32 ...
    sea_surface_temperature   (time, longitude, latitude) float32 ...
    ...                        ...
    total_cloud_cover  

Processing futures: 13it [00:00, 34.93it/s]
Processing futures: 2it [00:00,  4.16it/s]


1.06547 sec
======= RESULT SHAPE =======
result_dataset.shape:  torch.Size([7309, 99, 1450])


In [5]:
print(original[0][0][-1])

torch.Size([1450])


: 

In [14]:
%load_ext autoreload
%autoreload 2
from models.VariableAnalyzer.datasets.dataset import CustomDataset
dataset = CustomDataset(original[0], 4 * 7, device)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
print(dataset[3].shape)

torch.Size([29, 99, 1450])


In [17]:
from torch.utils.data import DataLoader, Dataset

dataloader = DataLoader(dataset, 32, shuffle=True)

In [18]:
print(len(dataloader))

228


In [19]:
for i, batch in enumerate(dataloader):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56


In [50]:
%load_ext autoreload
%autoreload 2
from models.VariableAnalyzer.models.model import VariableAnalyzer

model = VariableAnalyzer(100, 4 * 7, 1450, 10, 3, 3).to(device)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
for i, batch in enumerate(dataloader):
    print(model.get_time_seq(batch).)
    if i == 0:
        break

torch.Size([32, 2800])


In [26]:
def get_time_seq(src: torch.Tensor):
        time_seq = []
        for i in range(4 * 7):
            seq = [i for _ in range(100)]
            time_seq.extend(seq)
        return torch.tensor([time_seq for _ in range(src.size(0))])


for i, batch in enumerate(dataloader):
    print()
    if i == 0:
        break

(slice(0, 3, None), 1)


TypeError: list indices must be integers or slices, not tuple